<a href="https://colab.research.google.com/github/imperfect-geek/S-P-500-ETF-SImulation/blob/main/S%26P500_ETF_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter
!pip install yfinance

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import xlsxwriter
import math

[S&P 500 stocks list](https://drive.google.com/file/d/1BjSfAjS8s50ckaiLUHnPI2_iKq543CnC/view?usp=sharing)

In [ ]:
#stocks = pd.read_csv("sp_500_stocks.csv") #for utf-8
stocks = pd.read_csv("sp_500_stocks.csv",encoding='cp1252') #for windows platform encoding
stocks.head()

,Symbol,Security,GICS Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology


In [ ]:
symbol = "MSFT"
sample_data = yf.Ticker(symbol)
sample_data.info.keys()

dict_keys(['address1', 'city', 'state', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryDisp', 'sector', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'governanceEpochDate', 'compensationAsOfEpochDate', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'currency', 'enterpriseValue', 'profitMargins', 'floatSh

In [ ]:
f"""Current Price: { sample_data.info["currentPrice"] } || Market Cap: { sample_data.info["marketCap"] } || Floating Market Cap: { math.floor(sample_data.info["floatShares"]*sample_data.info["currentPrice"]) }"""

'Current Price: 328.39 || Market Cap: 2441740550144 || Floating Market Cap: 2439542198906'

In [ ]:
# columns = ["Ticker", "Stock Price", "Market Capitalization", "Free Float Market Cap", "Number of Shares to Buy"]
# df = pd.DataFrame(columns=columns)
# data_rows = []
# for ticker in stocks["Symbol"]:
#   data = yf.Ticker(ticker).info
#   ff_marketCap = math.floor(data["floatShares"]*data["currentPrice"]) if data.get("floatShares", 0) != 0 else data["marketCap"] #assumption
#   new_row = pd.DataFrame([ticker, data["currentPrice"], data["marketCap"], ff_marketCap, "NA"], index=columns).T
#   data_rows.append(new_row)

In [ ]:
tickers = " ".join(stocks["Symbol"])
tickers_obj = yf.Tickers(tickers)

In [ ]:
columns = ["Ticker", "Stock Price", "Market Capitalization", "Free Float Market Cap", "Number of Shares to Buy"]
df = pd.DataFrame(columns=columns)
data_rows = []
for ticker in stocks["Symbol"]:
  data = tickers_obj.tickers[ticker].info
  # CAT, EVRG, HPE, HPQ - Free float shares not available
  ff_marketCap = math.floor(data["floatShares"]*data["currentPrice"]) if data.get("floatShares", 0) != 0 else data["marketCap"] #assumption
  new_row = pd.DataFrame([ticker, data["currentPrice"], data["marketCap"], ff_marketCap, "NA"], index=columns).T
  data_rows.append(new_row)

In [ ]:
data_rows = pd.concat(data_rows)
df = pd.concat([df, data_rows]).reset_index(drop=True)

In [ ]:
df.tail()

,Ticker,Stock Price,Market Capitalization,Free Float Market Cap,Number of Shares to Buy
498,YUM,128.69,36044398592,35987083295,NA
499,ZBRA,262.57,13504054272,13406159372,NA
500,ZBH,127.34,26715676672,26520436321,NA
501,ZION,27.29,4041676288,3974051015,NA
502,ZTS,163.01,75328872448,75214400087,NA


In [ ]:
try:
  capital_available = float(input("Enter the value of your portfolio (in $):"))
except ValueError:
  print("That's not a number! \n Try again:")
  portfolio_size = float(input("Enter the value of your portfolio (in $):"))

Enter the value of your portfolio (in $):1000


In [ ]:
total_FF_marketCap = df["Free Float Market Cap"].sum()
for i in range(len(df)):
  percentage = (df.loc[i]["Free Float Market Cap"]/total_FF_marketCap) * 100
  df.loc[i]["Number of Shares to Buy"] = math.floor((percentage * capital_available) / df.loc[i]["Stock Price"])

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Free Float Market Capitalization', dollar_format],
                    'E': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 23, column_formats[column][1])

In [ ]:
writer.save()

<ipython-input-16-16d93d5523a4>:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
